Lambda School Data Science

*Unit 4, Sprint 2, Module 4*

---

# Neural Networks & GPUs (Prepare)
*aka Hyperparameter Tuning*

*aka Big Servers for Big Problems*

## Learning Objectives
* <a href="#p1">Part 1</a>: Describe the major hyperparemeters to tune
* <a href="#p2">Part 2</a>: Implement an experiment tracking framework
* <a href="#p3">Part 3</a>: Search the hyperparameter space using RandomSearch (Optional)

# Hyperparameter Options (Learn)
<a id="p1"></a>

## Overview

Hyperparameter tuning is much more important with neural networks than it has been with any other models that we have considered up to this point. Other supervised learning models might have a couple of parameters, but neural networks can have dozens. These can substantially affect the accuracy of our models and although it can be a time consuming process is a necessary step when working with neural networks.
​
Hyperparameter tuning comes with a challenge. How can we compare models specified with different hyperparameters if our model's final error metric can vary somewhat erratically? How do we avoid just getting unlucky and selecting the wrong hyperparameter? This is a problem that to a certain degree we just have to live with as we test and test again. However, we can minimize it somewhat by pairing our experiments with Cross Validation to reduce the variance of our final accuracy values.

### Load Boston Housing Data

In [1]:
import pandas as pd

url='https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv'

dataset = pd.read_csv(url, header=None)
dataset.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
X = dataset.values[:,0:8]
print (X.shape)
print (X)

(768, 8)
[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]
 [  1.     93.     70.    ...  30.4     0.315  23.   ]]


In [3]:
y = dataset.values[:,-1]
print (y.shape)
print (y)

(768,)
[1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0.
 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0.
 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Normalizing Input Data

It's not 100% necessary to normalize/scale your input data before feeding it to a neural network, the network can learn the appropriate weights to deal with data of as long as it is numerically represented,  but it is recommended as it can help **make training faster** and **reduces the chances that gradient descent might get stuck in a local optimum**.

<https://stackoverflow.com/questions/4674623/why-do-we-have-to-normalize-the-input-for-an-artificial-neural-network>

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
print(x_train[:10])

[[ 1.89917664e+00 -1.94750933e-01  1.55717838e+00 -1.32537379e+00
  -6.85859549e-01 -1.04293476e+00  1.60661512e+00  6.90585855e-02]
 [ 9.82865509e-01  6.40463359e-01 -5.54961074e-01  8.05525757e-01
   9.58734921e-01 -4.14806202e-01  6.26098151e-01  2.37289284e+00]
 [ 6.65543754e-02 -1.63817070e-01  1.12030334e-01 -5.50501227e-01
   6.71916029e-02 -1.29156898e+00 -2.95311052e-02  3.25040169e-01]
 [-8.49756758e-01  1.45521556e-01 -5.54961074e-01 -1.32537379e+00
  -6.85859549e-01 -2.44688051e-01 -3.63199566e-01  1.17831211e+00]
 [-2.38882669e-01 -1.33930385e+00  8.65099101e-04 -1.32537379e+00
  -6.85859549e-01  6.93762291e-02 -5.94425956e-01  4.95694558e-01]
 [-8.49756758e-01 -6.89692736e-01 -6.66126308e-01 -6.79646654e-01
  -6.85859549e-01 -8.59730598e-01  2.28037531e-01 -1.04019494e+00]
 [-8.49756758e-01  6.71397222e-01  7.79021741e-01  1.59798622e-01
   1.99742617e+00  1.36489138e+00  1.76466860e+00 -9.54867749e-01]
 [-8.49756758e-01  2.07389282e-01  1.00135221e+00  1.19296204e+00
   

### Model Validation using an automatic verification Dataset

Instead of doing seperate train test split class, Keras has a really nice feature that you can set the validation.data argument when fitting your model and Keras will take that portion of your test data and use it as a validation dataset. 

In [6]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Important Hyperparameters
inputs = x_train.shape[1]
epochs = 75
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(8, activation='relu', input_shape=(inputs,)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae', 'accuracy'])

# Fit Model
model.fit(x_train, y_train, 
          validation_data=(x_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Train on 514 samples, validate on 254 samples
Epoch 1/75
514/514 [==============================] - 2s 3ms/sample - loss: 0.4641 - mse: 0.4641 - mae: 0.5127 - accuracy: 0.6284 - val_loss: 0.3487 - val_mse: 0.3487 - val_mae: 0.4501 - val_accuracy: 0.6378
Epoch 2/75
514/514 [==============================] - 0s 539us/sample - loss: 0.3381 - mse: 0.3381 - mae: 0.4456 - accuracy: 0.6518 - val_loss: 0.2846 - val_mse: 0.2846 - val_mae: 0.4136 - val_accuracy: 0.6496
Epoch 3/75
514/514 [==============================] - 0s 546us/sample - loss: 0.2690 - mse: 0.2690 - mae: 0.4095 - accuracy: 0.6673 - val_loss: 0.2497 - val_mse: 0.2497 - val_mae: 0.4031 - val_accuracy: 0.6693
Epoch 4/75
514/514 [==============================] - 0s 566us/sample - loss: 0.2298 - mse: 0.2298 - mae: 0.3946 - accuracy: 0.6887 - val_loss: 0.2266 - val_mse: 0.2266 - val_mae: 0.3908 - val_accuracy: 0.6850
Epoch 5/75
514/514 [==============================] - 0s 570us/sample - loss: 0.2062 - mse: 0.2062 - mae: 0.3782 - a

Epoch 40/75
514/514 [==============================] - 0s 376us/sample - loss: 0.1405 - mse: 0.1405 - mae: 0.2924 - accuracy: 0.7938 - val_loss: 0.1882 - val_mse: 0.1882 - val_mae: 0.3432 - val_accuracy: 0.7362
Epoch 41/75
514/514 [==============================] - 0s 408us/sample - loss: 0.1407 - mse: 0.1407 - mae: 0.2979 - accuracy: 0.7996 - val_loss: 0.1878 - val_mse: 0.1878 - val_mae: 0.3437 - val_accuracy: 0.7283
Epoch 42/75
514/514 [==============================] - 0s 379us/sample - loss: 0.1401 - mse: 0.1401 - mae: 0.2932 - accuracy: 0.7918 - val_loss: 0.1869 - val_mse: 0.1869 - val_mae: 0.3404 - val_accuracy: 0.7323
Epoch 43/75
514/514 [==============================] - 0s 379us/sample - loss: 0.1399 - mse: 0.1399 - mae: 0.2952 - accuracy: 0.8035 - val_loss: 0.1864 - val_mse: 0.1864 - val_mae: 0.3403 - val_accuracy: 0.7362
Epoch 44/75
514/514 [==============================] - 0s 394us/sample - loss: 0.1399 - mse: 0.1399 - mae: 0.2942 - accuracy: 0.8016 - val_loss: 0.1859 - va

### Hyperparameter Tuning Approaches:

#### 1) Babysitting AKA "Grad Student Descent".

If you fiddled with any hyperparameters yesterday, this is basically what you did. This approach is 100% manual and is pretty common among researchers where finding that 1 exact specification that jumps your model to a level of accuracy never seen before is the difference between publishing and not publishing a paper. Of course the professors don't do this themselves, that's grunt work. This is also known as the fiddle with hyperparameters until you run out of time method.

#### 2) Grid Search

Grid Search is the Grad Student galaxy brain realization of: why don't I just specify all the experiments I want to run and let the computer try every possible combination of them while I go and grab lunch. This has a specific downside in that if I specify 5 hyperparameters with 5 options each then I've just created 5^5 combinations of hyperparameters to check. Which means that I have to train 3125 different versions of my model Then if I use 5-fold Cross Validation on that then my model has to run 15,525 times. This is the brute-force method of hyperparameter tuning, but it can be very profitable if done wisely. 

When using Grid Search here's what I suggest: don't use it to test combinations of different hyperparameters, only use it to test different specifications of **a single** hyperparameter. It's rare that combinations between different hyperparameters lead to big performance gains. You'll get 90-95% of the way there if you just Grid Search one parameter and take the best result, then retain that best result while you test another, and then retain the best specification from that while you train another. This at least makes the situation much more manageable and leads to pretty good results. 

#### 3) Random Search

Do Grid Search for a couple of hours and you'll say to yourself - "There's got to be a better way." Enter Random Search. For Random search you specify a hyperparameter space and it picks specifications from that randomly, tries them out, gives you the best results and says - That's going to have to be good enough, go home and spend time with your family. 

Grid Search treats every parameter as if it was equally important, but this just isn't the case, some are known to move the needle a lot more than others (we'll talk about that in a minute). Random Search allows searching to be specified along the most important parameter and experiments less along the dimensions of less important hyperparameters. The downside of Random search is that it won't find the absolute best hyperparameters, but it is much less costly to perform than Grid Search. 

#### 4) Bayesian Methods

One thing that can make more manual methods like babysitting and gridsearch effective is that as the experimenter sees results he can then make updates to his future searches taking into account the results of past specifications. If only we could hyperparameter tune our hyperparameter tuning. Well, we kind of can. Enter Bayesian Optimization. Neural Networks are like an optimization problem within an optimization problem, and Bayesian Optimization is a search strategy that tries to take into account the results of past searches in order to improve future ones. This is the most advanced method but can be a little bit tricky to implement, but there are some early steps with `hyperas` which is Bayesian optimization wrapper for `keras`. 

## What Hyperparameters are there to test?

- batch_size
- training epochs
- optimization algorithms
- learning rate
- momentum
- activation functions
- dropout regularization
- number of neurons in the hidden layer

There are more, but these are the most important.

## Follow Along

## Batch Size

Batch size determines how many observations the model is shown before it calculates loss/error and updates the model weights via gradient descent. You're looking for a sweet spot here where you're showing it enough observations that you have enough information to updates the weights, but not such a large batch size that you don't get a lot of weight update iterations performed in a given epoch. Feed-forward Neural Networks aren't as sensitive to bach_size as other networks, but it is still an important hyperparameter to tune. Smaller batch sizes will also take longer to train. 

In [7]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

dataset = pd.read_csv(url, header=None).values

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

/home/martin/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.6510416666666666 using {'batch_size': 20, 'epochs': 20}
Means: 0.6080729166666666, Stdev: 0.03737682316251907 with: {'batch_size': 10, 'epochs': 20}
Means: 0.6510416666666666, Stdev: 0.015072704300508107 with: {'batch_size': 20, 'epochs': 20}
Means: 0.5598958333333334, Stdev: 0.07105606067522006 with: {'batch_size': 40, 'epochs': 20}
Means: 0.52734375, Stdev: 0.07356482756159133 with: {'batch_size': 60, 'epochs': 20}
Means: 0.5455729166666666, Stdev: 0.03936514702560149 with: {'batch_size': 80, 'epochs': 20}
Means: 0.5885416666666666, Stdev: 0.05396974420849664 with: {'batch_size': 100, 'epochs': 20}


## Epochs

The number of training epochs has a large and direct affect on the accuracy, However, more epochs is almost always goign to better than less epochs. This means that if you tune this parameter at the beginning and try and maintain the same value all throughout your training, you're going to be waiting a long time for each iteration of GridSearch. I suggest picking a fixed moderat # of epochs all throughout your training and then Grid Searching this parameter at the very end. 

In [8]:
# define the grid search parameters
param_grid = {'batch_size': [20],
              'epochs': [20, 40, 60,200]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7083333333333334 using {'batch_size': 20, 'epochs': 200}
Means: 0.6705729166666666, Stdev: 0.016366933710906945 with: {'batch_size': 20, 'epochs': 20}
Means: 0.6419270833333334, Stdev: 0.03440063753547632 with: {'batch_size': 20, 'epochs': 40}
Means: 0.6861979166666666, Stdev: 0.029634913220679796 with: {'batch_size': 20, 'epochs': 60}
Means: 0.7083333333333334, Stdev: 0.010252614419286212 with: {'batch_size': 20, 'epochs': 200}


## Optimizer

Remember that there's a different optimizers [optimizers](https://keras.io/optimizers/). At some point, take some time to read up on them a little bit. "adam" usually gives the best results. The thing to know about choosing an optimizer is that different optimizers have different hyperparameters like learning rate, momentum, etc. So based on the optimizer you choose you might also have to tune the learning rate and momentum of those optimizers after that. 

## Learning Rate

Remember that the Learning Rate is a hyperparameter that is specific to your gradient-descent based optimizer selection. A learning rate that is too high will cause divergent behavior, but a Learning Rate that is too low will fail to converge, again, you're looking for the sweet spot. I would start out tuning learning rates by orders of magnitude: [.001, .01, .1, .2, .3, .5] etc. I wouldn't go above .5, but you can try it and see what the behavior is like. 

Once you have narrowed it down, make the window even smaller and try it again. If after running the above specification your model reports that .1 is the best optimizer, then you should probably try things like [.05, .08, .1, .12, .15] to try and narrow it down. 

It can also be good to tune the number of epochs in combination with the learning rate since the number of iterations that you allow the learning rate to reach the minimum can determine if you have let it run long enough to converge to the minimum. 

## Momentum

Momentum is a hyperparameter that is more commonly associated with Stochastic Gradient Descent. SGD is a common optimizer because it's what people understand and know, but I doubt it will get you the best results, you can try hyperparameter tuning its attributes and see if you can beat the performance from adam. Momentum is a property that decides the willingness of an optimizer to overshoot the minimum. Imagine a ball rolling down one side of a bowl and then up the opposite side a little bit before settling back to the bottom. The purpose of momentum is to try and escale local minima.

## Activation Functions

We've talked about this a little bit, typically you'l want to use ReLU for hidden layers and either Sigmoid, or Softmax for output layers of binary and multi-class classification implementations respectively, but try other activation functions and see if you can get any better results with sigmoid or tanh or something. There are a lot of activation functions that we haven't really talked about. Maybe you'll get good results with them. Maybe you won't. :) <https://keras.io/activations/>

## Network Weight Initialization

You saw how big of an effect the way that we initialize our network's weights can have on our results. There are **a lot** of what are called initialization modes. I don't understand all of them, but they can have a big affect on your model's initial accuracy. Your model will get further with less epochs if you initialize it with weights that are well suited to the problem you're trying to solve.

`init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']`

## Dropout Regularization and the Weight Constraint

the Dropout Regularization value is a percentage of neurons that you want to be randomly deactivated during training. The weight constraint is a second regularization parameter that works in tandem with dropout regularization. You should tune these two values at the same time. 

Using dropout on visible vs hidden layers might have a different effect. Using dropout on hidden layers might not have any effect while using dropout on hidden layers might have a substantial effect. You don't necessarily need to turn use dropout unless you see that your model has overfitting and generalizability problems.

## Neurons in Hidden Layer 

Remember that when we only had a single perceptron our model was only able to fit to linearly separable data, but as we have added layers and nodes to those layers our network has become a powerhouse of fitting nonlinearity in data. The larger the network and the more nodes generally the stronger the network's capacity to fit nonlinear patterns in data. The more nodes and layers the longer it will take to train a network, and higher the probability of overfitting. The larger your network gets the more you'll need dropout regularization or other regularization techniques to keep it in check. 

Typically depth (more layers) is more important than width (more nodes) for neural networks. This is part of why Deep Learning is so highly touted. Certain deep learning architectures have truly been huge breakthroughs for certain machine learning tasks. 

You might borrow ideas from other network architectures. For example if I was doing image recognition and I wasn't taking cues from state of the art architectures like resnet, alexnet, googlenet, etc. Then I'm probably going to have to do a lot more experimentation on my own before I find something that works.

There are some heuristics, but I am highly skeptical of them. I think you're better off experimenting on your own and forming your own intuition for these kinds of problems. 

- https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/

## Challenge
You will be expected to tune several hyperparameters in today's module project. 

# Experiment Tracking Framework (Learn)
<a id="p2"></a>

## Overview

You will notice quickly that managing the results of all the experiments you are running becomes challenging. Which set of parameters did the best? Are my results today different than my results yesterday? Although we use Ipython Notebooks to work, the format is not well suited to logging experimental results. Enter experiment tracking frameworks like [Comet.ml](https://comet.ml) and [Weights and Biases](https://wandb.ai/).

Those tools will help you track your experiments, store the results, and the code associated with those experiments. Experimental results can also be readily visualized to see changes in performance across any metric you care about. Data is sent to the tool as each epoch is completed, so you can also see if your model is converging. Let's check out Weights & Biases today. 

## Follow Along

Make sure you login into `wandb` in the terminal before running the next cell. 

In [14]:
import wandb
from wandb.keras import WandbCallback

In [15]:
wandb.init(project="boston", entity="lambda-ds7") #Initializes and Experiment

# Important Hyperparameters
X =  x_train
y =  y_train

inputs = X.shape[1]
wandb.config.epochs = 50
wandb.config.batch_size = 10

# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# Fit Model
model.fit(X, y, 
          validation_split=0.33, 
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
         )

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/martin/.netrc


Train on 344 samples, validate on 170 samples
Epoch 1/50
344/344 [==============================] - 1s 4ms/sample - loss: 0.2020 - mse: 0.2020 - mae: 0.3579 - val_loss: 0.1691 - val_mse: 0.1691 - val_mae: 0.3309
Epoch 2/50
344/344 [==============================] - 0s 504us/sample - loss: 0.1550 - mse: 0.1550 - mae: 0.3019 - val_loss: 0.1607 - val_mse: 0.1607 - val_mae: 0.3184
Epoch 3/50
344/344 [==============================] - 0s 388us/sample - loss: 0.1421 - mse: 0.1421 - mae: 0.2892 - val_loss: 0.1540 - val_mse: 0.1540 - val_mae: 0.3157
Epoch 4/50
344/344 [==============================] - 0s 317us/sample - loss: 0.1331 - mse: 0.1331 - mae: 0.2814 - val_loss: 0.1550 - val_mse: 0.1550 - val_mae: 0.3111
Epoch 5/50
344/344 [==============================] - 0s 340us/sample - loss: 0.1262 - mse: 0.1262 - mae: 0.2706 - val_loss: 0.1543 - val_mse: 0.1543 - val_mae: 0.3049
Epoch 6/50
344/344 [==============================] - 0s 558us/sample - loss: 0.1190 - mse: 0.1190 - mae: 0.2594 - v

344/344 [==============================] - 0s 572us/sample - loss: 0.0161 - mse: 0.0161 - mae: 0.0883 - val_loss: 0.2218 - val_mse: 0.2218 - val_mae: 0.3631
Epoch 50/50
344/344 [==============================] - 0s 555us/sample - loss: 0.0184 - mse: 0.0184 - mae: 0.0943 - val_loss: 0.2278 - val_mse: 0.2278 - val_mae: 0.3643


## Challenge

You will be expected to use Weights & Biases to try to tune your model during your module assignment today. 

# Hyperparameters with RandomSearchCV (Learn)

## Overview

Basically `GridSearchCV` takes forever. You'll want to adopt a slightly more sophiscated strategy.

## Follow Along

In [11]:
sweep_config = {
    'method': 'random',
    'parameters': {
        'learning_rate': {'distribution': 'normal'},
        'epochs': {'distribution': 'uniform',
                    'min': 100,
                    'max': 1000},
        'batch_size': {'distribution': 'uniform',
            'min': 10,
            'max': 400}
    }
}

In [12]:
sweep_id = wandb.sweep(sweep_config)

NameError: name 'wandb' is not defined

In [13]:
import wandb
from wandb.keras import WandbCallback
#Initializes and Experiment

from tensorflow.keras.optimizers import Adam

# Important Hyperparameters
X =  x_train
y =  y_train

inputs = X.shape[1]

def train():
    
    wandb.init(project="boston", entity="lambda-ds7") 
    
    config = wandb.config

    # Create Model
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(inputs,)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))

    # Optimizer 
    adam = Adam(learning_rate=config.learning_rate)

    # Compile Model
    model.compile(optimizer=adam, loss='mse', metrics=['mse', 'mae'])

    # Fit Model
    model.fit(X, y, 
              validation_split=0.33, 
              epochs=config.epochs, 
              batch_size=config.batch_size, 
              callbacks=[WandbCallback()]
             )

ModuleNotFoundError: No module named 'wandb'

In [ ]:
wandb.agent(sweep_id, function=train)

## Challenge

Try to apply RandomSearchCV to your module project today. 

# Review
* <a href="#p1">Part 1</a>: Describe the major hyperparemeters to tune
    - Activation Functions
    - Optimizer
    - Number of Layers
    - Number of Neurons
    - Batch Size
    - Dropout Regulaization
    - Learning Rate
    - Number of Epochs
    - and many more
* <a href="#p2">Part 2</a>: Implement an experiment tracking framework
    - Weights & Biases
    - Comet.ml
    - By Hand / GridSearch
* <a href="#p3">Part 3</a>: Search the hyperparameter space using RandomSearch
    - Sklearn still useful (haha)
    - Integration with Wieghts & Biases
* <a href="#p4">Part 4</a>: Discuss emerging hyperparameter tuning strategies
    - Bayesian Optimization
    - Hyperopt
    - Genetic Evolution

# Sources

## Additional Reading
- https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
- https://blog.floydhub.com/guide-to-hyperparameters-search-for-deep-learning-models/
- https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/
- https://machinelearningmastery.com/introduction-to-weight-constraints-to-reduce-generalization-error-in-deep-learning/
- https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/